In [ ]:
%matplotlib inline
from ROOT import TFile, TTree

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import re

from root_numpy import root2array

In [ ]:
filedir = '/Users/davidkaleko/larlite/UserDev/KalekoAna/CCInclusive/mac'
!ls $filedir | grep ana

In [ ]:
allfile = filedir+'/FluxAna_ana_out_allbnbcosmic.root'
cutfile = '/Users/davidkaleko/Data/larlite/080116_selection_output/'
cutfile += 'ana_out_trkpandoraNuPMA_vtxpmtrack_mcc71ext2bnb3_1.root'

df_dict = {}
df_dict['all'] = pd.DataFrame( root2array ( allfile, 'fluxtree' ) )
df_dict['cut'] = pd.DataFrame( root2array ( cutfile, 'tree' ) )

In [ ]:
print df_dict['all'].columns.values
print df_dict['cut'].columns.values

       Decay type	Decay code
       
       K0L -> nue pi- e+	1
       K0L -> nuebar pi+ e-	2
       K0L -> numu pi- mu+	3
       K0L -> numubar pi+ mu-	4
       K+ -> numu mu+	        5
       K+ -> nue pi0 e+	        6
       K+ -> numu pi0 mu+	7
       K- -> numubar mu-	8
       K- -> nuebar pi0 e-	9
       K- -> numubar pi0 mu-	10
       mu+ -> numubar nue e+	11
       mu- -> numu nuebar e-	12
       pi+ -> numu mu+	        13
       pi- -> numubar mu-	14

In [ ]:
def kaon_2d_plot(mydf,myquery,mytitle):
    plt.figure(figsize=(10,6))
    myy = mydf.query(myquery)['kaon_prod_E'].values
    myx = mydf.query(myquery)['kaon_prod_theta'].values
    bins = (np.linspace(0,12,20),np.linspace(0,8,20))
    poop = plt.hist2d(x=myx,y=myy,bins=bins,cmin=1)
    plt.colorbar()
    plt.grid(True)
    plt.xlabel('K+ Production Angle (degrees)',fontsize=16)
    plt.ylabel('K+ Production Energy [GeV]',fontsize=16)
    plt.title(mytitle,fontsize=16)

def kaon_1d_plots(myquery,mytitle):
    
    #angle plot
    plt.figure(figsize=(10,6))
    bins = np.linspace(0,10,20)
    
    thisquery = myquery + ' and nu_in_fidvol'
    myx = df_dict['all'].query(thisquery)['kaon_prod_theta'].values
    mymean, myrms = np.mean(myx), np.std(myx)
    poop = plt.hist(myx,bins=bins,color='r',histtype='step',
                    label='All (Mean %0.2f, RMS %0.2f)' % (mymean,myrms))
    
    thisquery = myquery + ' and correct_ID == 1'
    myx = df_dict['cut'].query(thisquery)['kaon_prod_theta'].values
    mymean, myrms = np.mean(myx), np.std(myx)
    poop = plt.hist(myx,bins=bins,color='b',histtype='step',
                    label='Selected (Mean %0.2f, RMS %0.2f)' % (mymean,myrms))
    
    thisquery = myquery + ' and correct_ID == 1 and nu_E_estimate > 2.5'
    myx = df_dict['cut'].query(thisquery)['kaon_prod_theta'].values
    mymean, myrms = np.mean(myx), np.std(myx)
    poop = plt.hist(myx,bins=bins,color='g',histtype='step',
                    label='Selected > 2.5GeV (Mean %0.2f, RMS %0.2f)' % (mymean,myrms))
    
    plt.grid(True)
    plt.xlabel('K+ Production Angle [degrees]',fontsize=16)
    plt.legend(loc='best')
    plt.title(mytitle,fontsize=16)
    
    #Energy plot
    plt.figure(figsize=(10,6))
    bins = np.linspace(0,8,20)
    
    thisquery = myquery + ' and nu_in_fidvol'
    myx = df_dict['all'].query(thisquery)['kaon_prod_E'].values
    mymean, myrms = np.mean(myx), np.std(myx)
    poop = plt.hist(myx,bins=bins,color='r',histtype='step',
                    label='All (Mean %0.2f, RMS %0.2f)' % (mymean,myrms))
    
    thisquery = myquery + ' and correct_ID == 1'
    myx = df_dict['cut'].query(thisquery)['kaon_prod_E'].values
    mymean, myrms = np.mean(myx), np.std(myx)
    poop = plt.hist(myx,bins=bins,color='b',histtype='step',
                    label='Selected (Mean %0.2f, RMS %0.2f)' % (mymean,myrms))
    
    thisquery = myquery + ' and correct_ID == 1 and nu_E_estimate > 2.5'
    myx = df_dict['cut'].query(thisquery)['kaon_prod_E'].values
    mymean, myrms = np.mean(myx), np.std(myx)
    poop = plt.hist(myx,bins=bins,color='g',histtype='step',
                    label='Selected > 2.5GeV (Mean %0.2f, RMS %0.2f)' % (mymean,myrms))
    
    plt.grid(True)
    plt.xlabel('K+ Production Energy [GeV]',fontsize=16)
    plt.legend(loc='best')
    plt.title(mytitle,fontsize=16)

In [ ]:
# Numu from kaon decay in cryostat: all events
mydf = df_dict['all']
mybasequery = '(fndecay == 3 or fndecay == 4 or fndecay == 5 or fndecay == 7 or fndecay == 8 or fndecay == 10)'
mybasequery += ' and (true_nu_pdg == 14 or true_nu_pdg == -14)'
myquery = mybasequery
myquery += ' and nu_in_fidvol'
mytitle = 'All numu from Kaon Decay Fiducial Volume'
kaon_2d_plot(mydf,myquery,mytitle)

# Numu from kaon decay in crystat: selected events
mydf = df_dict['cut']
myquery = mybasequery
myquery += ' and correct_ID == 1'
mytitle = 'Selected numu from Kaon Decay Fiducial Volume'
kaon_2d_plot(mydf,myquery,mytitle)

# Numu from kaon decay in crystat: selected events after nu_E_estimate cut
mydf = df_dict['cut']
myquery = mybasequery
myquery += ' and nu_E_estimate > 2.5'
myquery += ' and correct_ID == 1'
mytitle = 'Selected numu from Kaon Decay Fiducial Volume: After 2.5GeV Reco E Cut'
kaon_2d_plot(mydf,myquery,mytitle)


mytitle = 'Numu from Kaon Decay in the Fiducial Volume'
kaon_1d_plots(mybasequery,mytitle)

In [ ]:
# Nue from kaon decay in cryostat: all events
mybasequery = '(fndecay == 1 or fndecay == 2 or fndecay == 6 or fndecay == 9)'
mybasequery += ' and (true_nu_pdg == 12 or true_nu_pdg == -12)'
mydf = df_dict['all']
myquery = mybasequery
myquery += ' and nu_in_fidvol'
mytitle = 'All nue from Kaon Decay Fiducial Volume'
kaon_2d_plot(mydf,myquery,mytitle)

# Nue from kaon decay in crystat: selected events
mydf = df_dict['cut']
myquery = mybasequery
myquery += ' and correct_ID == 1'
mytitle = 'Selected nue from Kaon Decay Fiducial Volume'
kaon_2d_plot(mydf,myquery,mytitle)

# Nue from kaon decay in crystat: selected events after nu_E_estimate cut
mydf = df_dict['cut']
myquery = mybasequery
myquery += ' and nu_E_estimate > 2.5'
myquery += ' and correct_ID == 1'
mytitle = 'Selected nue from Kaon Decay Fiducial Volume: After 2.5GeV Reco E Cut'
kaon_2d_plot(mydf,myquery,mytitle)


mytitle = 'Nue from Kaon Decay in the Fiducial Volume'
kaon_1d_plots(mybasequery,mytitle)